## **Function Calling LLMs - Team Project**

In [30]:
import openai
import json
import os

from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("API_KEY")

## Data

## Functions

In [40]:
from functools import wraps
from pprint import pprint
from registry import Registry


register = Registry()

def to_json(fcn):
    @wraps(fcn)
    def wrapper(*args, **kwargs):
        result = fcn(*args, **kwargs)
        return json.dumps(result)
    return wrapper

@register.register
@to_json
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": 72,
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return weather_info

pprint(register.registered_functions)

{'get_current_weather': <function get_current_weather at 0x000002202E5AA3E0>}


In [26]:
FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    }
]

## LLM

In [27]:
from enum import Enum
from typing import Union

class Role(Enum):
    ASSISTANT = "assistant"
    FUNCTION = "function"
    SYSTEM = "system"
    USER = "user"

class Model_Version(Enum):
    GPT3 = "gpt-3.5-turbo-0613"
    GPT4 = "gpt-4-0613"

CONTEXT = {"role": Role.SYSTEM.value, "content": "Answer in one very short sentence."}

MAX_ITER = 3

MODEL_VERSION = Model_Version.GPT3.value


In [64]:
outer = []


def _add_message_to_context(role:Role, content:str, messages:list, function_call:dict=None) -> None:
    if role == Role.ASSISTANT:
        d = {"role": Role.ASSISTANT.value, "content": content}
        if function_call is not None: d |= {"function_call": function_call}
        messages.append(d)

    if role == Role.USER:
        messages.append({"role": Role.USER.value, "content": content})
    
    if role == Role.FUNCTION:
        messages.append({"role": Role.FUNCTION.value, "name": function_call["name"], "content": content})
        

def send_message(role:Union[Role.USER, Role.FUNCTION], content:str, messages:list, function_call:dict=None) -> dict:
    
    _add_message_to_context(role, content, messages, function_call)
    
    response = openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=messages,
        function_call="auto",
        functions=FUNCTIONS,
    )
    
    response = response["choices"][0]["message"]
    response_message = response["content"]
    
    is_function_call = response.get("function_call")
    if is_function_call:
        
        outer.append(is_function_call)
        
        _add_message_to_context(Role.ASSISTANT, response_message, messages, is_function_call.to_dict())
    
    return response


def handle_function(function:dict) -> json:
    function_name, function_args = function["name"], json.loads(function["arguments"])
    
    if function_name in register.registered_functions:
        function_to_call = register[function_name]
        result = function_to_call(**function_args)
        return result

def run(user_question:str, verbose=False):    
    messages = [CONTEXT]
    
    # Step 1: send the conversation (context) and available functions to GPT
    response = send_message(Role.USER, user_question, messages)

    print(response)
    
    print(messages)
    
    current_iteration = 1
    while (current_iteration <= MAX_ITER):
    
        # Step 2: check if GPT wanted to call a function 
        function_call = response.get("function_call")
        
        if not function_call:
            break
        
        print(function_call)
    
        if verbose: print("function-call detected...\n")
        # Step 3: call the function
        function_result = handle_function(function_call)
        
        print(function_result)
        
        # Step 4: send the info on the function call and function response to GPT
        response = send_message(Role.FUNCTION, function_result, messages, function_call)
        
        print(response)
        
    if verbose:
        print(messages)
    
    print(response)
    
    return response["content"]

In [65]:
response = run("What is the current weather in San Francisco, CA?", verbose=True)
print(response)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
  }
}
[{'role': 'system', 'content': 'Answer in one very short sentence.'}, {'role': 'user', 'content': 'What is the current weather in San Francisco, CA?'}, {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "San Francisco, CA"\n}'}}]
{
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
}
function-call detected...

{"location": "San Francisco, CA", "temperature": 72, "unit": "fahrenheit", "forecast": ["sunny", "windy"]}
{
  "role": "assistant",
  "content": "The current weather in San Francisco, CA is sunny and windy with a temperature of 72\u00b0F."
}
[{'role': 'system', 'content': 'Answer in one very short sentence.'}, {'role': 'user', 'content': 'What is the current weather in San Francisco, CA?'}, {'role': 'assi

NameError: name 'response_message' is not defined

## Benchmark

In [58]:
obj = outer[0]

{'name': 'get_current_weather',
 'arguments': '{\n  "location": "San Francisco, CA"\n}'}